In [2]:
from mx.elemwise_ops import quantize_elemwise_op
from mx.mx_ops import quantize_mx_op
from mx.specs import MxSpecs

In [3]:

def mx_quant_dequant(input, mx_specs, axes=-1):
    dtype = input.dtype
    # bf_in = quantize_elemwise_op(
    #     input.float(), mx_specs=mx_specs, round=mx_specs["round_output"]
    # )
    qin = quantize_elemwise_op(
        input,
        mx_specs,
        round=mx_specs["round_mx_output"],
    )
    qin = qin.to(dtype)
    return qin

In [4]:
mx_specs = MxSpecs()
mx_specs['custom_cuda'] = True
mx_specs['w_elem_format'] = 'int4'
mx_specs['w_scale_mode'] = 2
mx_specs['per_tensor'] = False
mx_specs['scale_bits'] = 16
mx_specs['block_size'] = 4

In [5]:
import torch
tensor = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 400, -1]
                    #   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 400, -1],
                    #   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 400, -1],
                    #   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 400, -1],
                     , dtype=torch.float32)
quantized_tensor = mx_quant_dequant(tensor.unsqueeze(0), mx_specs, axes=-1)
print(quantized_tensor)

tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 400.,  -1.]])


In [6]:
tensor = tensor.reshape(3, 4)
q_t = (tensor / (tensor.max(dim=-1, keepdim=True)[0] / 7)).round().clamp(-8, 7) * (tensor.max(dim=-1, keepdim=True)[0] / 7)
print(q_t)

tensor([[  0.0000,   0.8571,   2.1429,   3.0000],
        [  4.0000,   5.0000,   6.0000,   7.0000],
        [  0.0000,   0.0000, 400.0000,  -0.0000]])


In [7]:
for k, v in mx_specs.items():
    print(f"{k}: {v}")

scale_bits: 16
w_elem_format: int4
a_elem_format: None
A_elem_format: None
B_elem_format: None
w_elem_format_bp: None
a_elem_format_bp_ex: None
a_elem_format_bp_os: None
mx_flush_fp32_subnorms: False
shared_exp_method: max
block_size: 4
bfloat: 0
fp: 0
bfloat_subnorms: True
quantize_backprop: True
round: nearest
round_m: nearest
round_weight: nearest
round_output: nearest
round_grad_weight: nearest
round_grad_input: nearest
round_mx_output: nearest
round_mx_input_grad_input: nearest
round_mx_weight_grad_input: nearest
round_mx_grad_output_grad_input: nearest
round_mx_input_grad_weight: nearest
round_mx_grad_output_grad_weight: nearest
softmax_exp2: False
vec_use_exp2: False
vec_use_recip: False
custom_cuda: True
a_scale_mode: 0
w_scale_mode: 2
A_scale_mode: 0
B_scale_mode: 0
per_tensor: False
